In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import time
from datetime import datetime
import tensorflow_addons as tfa
import keras_tuner as kt

In [2]:
def exports():   
    # Set CUDA and CUPTI paths  
    os.environ['CUDA_HOME'] = '/usr/local/cuda'
    os.environ['PATH']= '/usr/local/cuda/bin:$PATH'  
    os.environ['CPATH'] = '/usr/local/cuda/include:$CPATH'  
    os.environ['LIBRARY_PATH'] = '/usr/local/cuda/lib64:$LIBRARY_PATH'  
    os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/extras/CUPTI/lib64:$LD_LIBRARY_PATH'  
    os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:$LD_LIBRARY_PATH'
    os.environ['TF_GPU_THREAD_MODE']='gpu_private'

In [3]:
exports

<function __main__.exports()>

Load data

In [4]:
cf15= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/cf15.csv', low_memory=False, header=None)
gf25= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gf25.csv', low_memory=False, header=None)
gf34= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gf34.csv', low_memory=False, header=None)
gm01= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gm01.csv', low_memory=False, header=None)
hc10= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/hc10.csv', low_memory=False, header=None)
nc46= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nc46.csv', low_memory=False, header=None)
nc74= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nc74.csv', low_memory=False, header=None)
nf11= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nf11.csv', low_memory=False, header=None)
no22= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/no22.csv', low_memory=False, header=None)
wc16= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wc16.csv', low_memory=False, header=None)
wc41= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wc41.csv', low_memory=False, header=None)
wo27= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo27.csv', low_memory=False, header=None)
wo34= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo34.csv', low_memory=False, header=None)
wo36= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo36.csv', low_memory=False, header=None)
wo37= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo37.csv', low_memory=False, header=None)
wo40= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo40.csv', low_memory=False, header=None)
wo45= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo45.csv', low_memory=False, header=None)

In [5]:
val_1 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gm01_EarlyMarch.csv', low_memory=False, header=None)
val_2 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo36_EarlyMarch.csv', low_memory=False, header=None)
val_3 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/go22.csv', low_memory=False, header=None)

In [25]:
frames =[cf15, gf25,gm01,nc46, nc74,nf11, wc16, wc41,wo27,wo34, wo36, wo37, wo40, wo45]
frames_2=[val_1, val_2, val_3]
data = pd.concat(frames)
val = pd.concat(frames_2)
val=val_2

Define data cleaning functions

In [20]:
def snip(data):
    #Snip unwanted characters 

    itime= data.iloc[:,0]
    itime=itime.map(lambda x: str(x[6:]))
    itime=itime.map(lambda x: float(x))

    date=itime
    minute=date.map(lambda x: time.gmtime(x+7200)[4])
    hour=date.map(lambda x: time.gmtime(x+7200)[3])
    day=date.map(lambda x: time.gmtime(x+7200)[2])
    month=date.map(lambda x: time.gmtime(x+7200)[1])

    interface= data.iloc[:,-14]
    interface=interface.map(lambda x: str(x))
    interface=interface.map(lambda x: str(x[15:]))
    interface=interface.map(lambda x: str(x[:-1]))

    dev_id=data.iloc[:,3]
    dev_id=dev_id.map(lambda x: str(x))    
    dev_id=dev_id.map(lambda x: str(x[7:]))
    dev_id=dev_id.map(lambda x: str(x[:-1]))

    jitter=data.iloc[:, -13]
    jitter=jitter.map(lambda x: str(x))
    jitter=jitter.map(lambda x: str(x[8:]))
    jitter=jitter.map(lambda x: str(x[:-1]))
    jitter=jitter.map(lambda x: float(x))

    latency= data.iloc[:,-12]
    latency=latency.map(lambda x: str(x))
    latency=latency.map(lambda x: str(x[9:]))
    latency=latency.map(lambda x: str(x[:-1]))
    latency=latency.map(lambda x: float(x))

    inband= data.iloc[:,-15]
    inband=inband.map(lambda x: str(x))
    inband=inband.map(lambda x: str(x[22:]))
    inband=inband.map(lambda x: str(x[:-5]))
    inband=inband.map(lambda x: float(x))

    outband= data.iloc[:, -5]
    outband=outband.map(lambda x: str(x))
    outband=outband.map(lambda x: x.split('"')[1])
    outband=outband.map(lambda x: str(x[:-4]))
    outband=outband.map(lambda x: float(x))

    packet=data.iloc[:,-4]
    packet=packet.map(lambda x: str(x))
    packet=packet.map(lambda x: str(x[12:]))
    packet=packet.map(lambda x: str(x[:-2]))
    packet=packet.map(lambda x: float(x))

    status=data.iloc[:,-2]
    status=status.map(lambda x: str(x))
    status=status.map(lambda x: str(x[8:]))
    status=status.map(lambda x: str(x[:-1]))

    status[:] = np.where(status=="up", 1,0)
    status=status.map(lambda x: float(x))
    status=status.rolling(20).mean()
    status=status.map(np.floor)

    train_init=pd.concat([itime, minute, hour, day, month, interface, dev_id, inband, outband, latency, packet, jitter, status], axis=1)
    train_init.columns=['Time','Minute', 'Hour','Day','Month', 'Interface', 'Dev_id', 'Inband', 'Outband', 'Latency', 'Packet', 'Jitter', 'Status']

    #Dedupp based on interfaces
    train=train_init[train_init.Interface != 'NI2A-WWW1']
    train=train[train.Interface != 'NI2B-WWW1']
    train=train[train.Interface != 'NI2A-WWW2']
    train=train[train.Interface != 'NI2B-WWW2']
    train=train[train.Interface != 'NI2A-WWW3']    
    train=train[train.Interface != 'NI2B-WWW3']
    train=train[train.Interface != 'NI2A-MPLS']
    train['Interface']= train['Interface'].astype('category')
    train['Interface']=train['Interface'].cat.codes

    #Remove unwanted columns
    train_label=train.filter(['Status'])
    train=train.drop(['Status','Inband', 'Outband'], axis=1)
    train= train.sort_values(by=['Dev_id', 'Time'])
    train=train.reset_index(drop=True)
    train=train.drop(['Dev_id'], axis=1)

    return train, train_label

In [8]:
train,train_label=snip(data)
train=train.iloc[:-700, :]
train=train.to_numpy()
train_label=train_label.iloc[700:]

In [26]:
test,test_label=snip(val)
test=test.iloc[:-700, :]
test=test.to_numpy()
test_label=test_label.iloc[700:]

In [27]:
#train, train_label = preproccess(data)
mean=train.mean(axis=0)
train-=mean
std = train.std(axis=0)
train/=std
#test, test_label = preproccess(val)
test-=mean
test/=std

train_dataset = tf.data.Dataset.from_tensor_slices((train, train_label))
test_dataset = tf.data.Dataset.from_tensor_slices((test, test_label))

In [11]:
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')

2022-03-13 00:36:23.357866: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-03-13 00:36:23.357983: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-03-13 00:36:23.439290: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1630] Profiler found 1 GPUs
2022-03-13 00:36:23.444424: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory
2022-03-13 00:36:23.481588: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-03-13 00:36:23.481716: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed


Build and train model

In [17]:
def get_model():
    model = tf.keras.Sequential([
            tf.keras.layers.Dense(80, activation='relu',kernel_regularizer='l1_l2'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(80, activation='relu',kernel_regularizer='l1_l2'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')])
    model.compile(optimizer='adam',
            loss='BinaryCrossentropy',
            metrics=tfa.metrics.F1Score(num_classes=1, average='micro'))
    return model

In [23]:
model = get_model()       
history = model.fit(train, train_label, epochs=5, batch_size=128,  callbacks = [tboard_callback])

Epoch 1/5
 506/5394 [=>............................] - ETA: 38s - loss: 1.7759 - f1_score: 0.7776

2022-03-13 00:46:44.622260: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-03-13 00:46:44.622293: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.


 517/5394 [=>............................] - ETA: 38s - loss: 1.7524 - f1_score: 0.7774

2022-03-13 00:46:44.964633: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-03-13 00:46:44.964976: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-03-13 00:46:45.040608: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:521]  GpuTracer has collected 3084 callback api events and 3038 activity events. 
2022-03-13 00:46:45.089470: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-03-13 00:46:45.164916: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/20220313-003623/plugins/profile/2022_03_13_00_46_45

2022-03-13 00:46:45.286761: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/20220313-003623/plugins/profile/2022_03_13_00_46_45/jordan-desktop.trace.json.gz


 534/5394 [=>............................] - ETA: 43s - loss: 1.7179 - f1_score: 0.7773

2022-03-13 00:46:45.388111: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/20220313-003623/plugins/profile/2022_03_13_00_46_45

2022-03-13 00:46:45.394162: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to logs/20220313-003623/plugins/profile/2022_03_13_00_46_45/jordan-desktop.memory_profile.json.gz
2022-03-13 00:46:45.395465: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: logs/20220313-003623/plugins/profile/2022_03_13_00_46_45
Dumped tool data for xplane.pb to logs/20220313-003623/plugins/profile/2022_03_13_00_46_45/jordan-desktop.xplane.pb
Dumped tool data for overview_page.pb to logs/20220313-003623/plugins/profile/2022_03_13_00_46_45/jordan-desktop.overview_page.pb
Dumped tool data for input_pipeline.pb to logs/20220313-003623/plugins/profile/2022_03_13_00_46_45/jordan-desktop.input_pipeline.pb
Dumped tool data for tensorflow_stats.pb to logs/2022

5394/5394 [==============================] - 36s 7ms/step - loss: 0.7702 - f1_score: 0.7753
Epoch 2/5
5394/5394 [==============================] - 34s 6ms/step - loss: 0.6657 - f1_score: 0.7753
Epoch 3/5
5394/5394 [==============================] - 33s 6ms/step - loss: 0.6655 - f1_score: 0.7753
Epoch 4/5
5394/5394 [==============================] - 33s 6ms/step - loss: 0.6654 - f1_score: 0.7753
Epoch 5/5
5394/5394 [==============================] - 33s 6ms/step - loss: 0.6654 - f1_score: 0.7753


In [14]:
%load_ext tensorboard

In [15]:
%tensorboard --logdir logs

In [28]:
test_loss, test_acc =model.evaluate(test, test_label, verbose=2)
print('\nTest Accuracy:', test_acc)

1447/1447 - 4s - loss: 0.2956 - f1_score: 0.9994 - 4s/epoch - 2ms/step

Test Accuracy: 0.9993516206741333


model.save('model.h5')

1:31 after loading datasets